In [1]:
import numpy as np   # basic numeric module in python, for array and matrix computation
import pandas as pd  # advanced numeric module, excels in data frame analysis
import matplotlib.pyplot as plt  # for data visualization
%pylab inline
import statsmodels.formula.api as smf    # for OLS regression

Populating the interactive namespace from numpy and matplotlib


## Example. Income vs Education in NYC zip code areas

Model average income per capita against percentages of individuals of different education level within the area

### Memo:
* **IncomePerCapita**----measured in USD
* **PopOver25** et al----population number under each category, e.g.
    * total population over 25 years old
    * holding a Bachelor's degree
    * graduating from professional school, etc.

In [2]:
data4 = pd.read_csv('data/IncomeEduReg.csv')
data4.head()

,Zipcode,IncomePerCapita,PopOver25,LessThanHS,HighSchool,SomeCollege,Bachelor,Master,Doctorate,ProfSchool
0,10001,77512.0,16328,1389,1665,2075,6061,3412,519,1207
1,10002,26905.0,60932,21170,12718,8532,12721,4001,641,1149
2,10003,79088.0,41182,1499,2810,4516,17958,9094,1626,3679
3,10004,98020.0,2279,29,87,305,984,550,86,238
4,10005,99633.0,5954,133,103,454,2745,1637,219,663


Starting from now we denote Income per capita by IPC:
$$IPC = \frac{Total \: Income}{Total \: Population}$$
But Total Income can be calculated as
$$ Total\: Income = \sum_k Total \: Income \:in\: Category_k$$
$$ = \sum_k (IPC \: within \: Category_k  \times Population \: of\: Category_k)$$
Then, IPC can be rewritten as
$$ \sum_k (IPC \: within \: Category_k  \times \frac{Population \: of\: Category_k}{Total \: Population})$$

$$= \sum_{k} (I_k \times p_k)$$

where k is the category index, $I_k$ is the average income within category k, and $p_k$ is the population percentage of category k.  
Our goal is to fit these $I_k$ as regression coefficients, note that since all these percentages sum up to 100%, we can omit one last $p_k$ and rewrite this term as 1 minus the rest. For example, if there are 3 categories in total, we have:
$$p_1 + p_2 + p_3 = 1$$which means$$p_3 = 1 - p_1 - p_2$$
Hence  $$IPC = I_1p_1 + I_2p_2 + I_3p_3$$ $$= I_1p_1 + I_2p_2 + I_3(1 - p_1 -p_2)$$ $$= I_3 + (I_1 - I_3)p_1 + (I_2 - I_3)p_2$$
which means we are equivalently fitting $I_3, (I_1 - I_3), (I_2 - I_3)$ rather than $I_1, I_3, I_3$, this is also where the intercept ($I_3$) comes from.

In [3]:
data4.dropna(inplace = True)  #drop NAN to avoid invalid computation
data4 = pd.concat([data4.IncomePerCapita,      # Convert unit to 1k USD, only for scaling purpose
        data4.iloc[:,3:].div(data4.PopOver25, axis = 0)],  # Compute the percentage, column-wise
        axis = 1)  # Concatenate the Income column with percentages, row-wise

In [4]:
# Run the regression of income per capita against education percentages (leave out "LessThanHS")

In [5]:
# Here's another shortcut or "trick" for string manipulation:
# For those strings with repetitive pattern, we can use str.join functions to concatenate them
# First, check the column names
data4.columns

Index([u'IncomePerCapita', u'LessThanHS', u'HighSchool', u'SomeCollege',
       u'Bachelor', u'Master', u'Doctorate', u'ProfSchool'],
      dtype='object')

In [6]:
data4.head()

,IncomePerCapita,LessThanHS,HighSchool,SomeCollege,Bachelor,Master,Doctorate,ProfSchool
0,77512.0,0.085069,0.101972,0.127082,0.371203,0.208966,0.031786,0.073922
1,26905.0,0.347436,0.208724,0.140025,0.208774,0.065663,0.010520,0.018857
2,79088.0,0.036399,0.068234,0.109660,0.436064,0.220825,0.039483,0.089335
3,98020.0,0.012725,0.038175,0.133831,0.431768,0.241334,0.037736,0.104432
4,99633.0,0.022338,0.017299,0.076251,0.461035,0.274941,0.036782,0.111354


In [7]:
# Select the ones you want, then put them in your formula
' + '.join(data4.columns[2:8])

'HighSchool + SomeCollege + Bachelor + Master + Doctorate + ProfSchool'

In [8]:
lm2 = smf.ols(formula = 'IncomePerCapita ~ ' + ' + '.join(data4.columns[2:8]), data = data4).fit()
print(lm2.summary())

                            OLS Regression Results                            
Dep. Variable:        IncomePerCapita   R-squared:                       0.885
Model:                            OLS   Adj. R-squared:                  0.881
Method:                 Least Squares   F-statistic:                     223.0
Date:                Tue, 01 Oct 2019   Prob (F-statistic):           6.25e-79
Time:                        10:58:16   Log-Likelihood:                -1925.6
No. Observations:                 181   AIC:                             3865.
Df Residuals:                     174   BIC:                             3888.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -7425.3552   8555.679     -0.868      

Diagnoze muticollinearity

In [9]:
#so what if we just keep the ones with low p-value?
lm2_1 = smf.ols(formula = 'IncomePerCapita ~ Bachelor + ProfSchool', data = data4).fit()
print(lm2_1.summary())

                            OLS Regression Results                            
Dep. Variable:        IncomePerCapita   R-squared:                       0.883
Model:                            OLS   Adj. R-squared:                  0.882
Method:                 Least Squares   F-statistic:                     673.3
Date:                Tue, 01 Oct 2019   Prob (F-statistic):           9.72e-84
Time:                        10:58:16   Log-Likelihood:                -1926.9
No. Observations:                 181   AIC:                             3860.
Df Residuals:                     178   BIC:                             3869.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3196.1341   2350.043      1.360      0.1

In [10]:
data4.corr()

,IncomePerCapita,LessThanHS,HighSchool,SomeCollege,Bachelor,Master,Doctorate,ProfSchool
IncomePerCapita,1.000000,-0.711450,-0.825741,-0.677916,0.871596,0.882306,0.729626,0.932823
LessThanHS,-0.711450,1.000000,0.523175,0.225423,-0.788850,-0.802496,-0.568103,-0.702364
HighSchool,-0.825741,0.523175,1.000000,0.709713,-0.874121,-0.874972,-0.757934,-0.876771
SomeCollege,-0.677916,0.225423,0.709713,1.000000,-0.682267,-0.663155,-0.644476,-0.715817
Bachelor,0.871596,-0.788850,-0.874121,-0.682267,1.000000,0.920058,0.706647,0.874991
Master,0.882306,-0.802496,-0.874972,-0.663155,0.920058,1.000000,0.793292,0.918050
Doctorate,0.729626,-0.568103,-0.757934,-0.644476,0.706647,0.793292,1.000000,0.782667
ProfSchool,0.932823,-0.702364,-0.876771,-0.715817,0.874991,0.918050,0.782667,1.000000


#### We can see that these variables are highly correlated to each other, so the results' interpretability is suffering from multicollinearity.

## In-class excercise

Q1. Perform regression vs Bachelor and "Advanced" which incorporated Master, Doctorate and ProfSchool

Q2. Visualize the regression fit by plotting the observation versus our prediction for the income per capita